In [7]:
import numpy as np
import tensorflow as tf

In [8]:
with open("1268-0.txt", "r", encoding="utf-8") as f:
    text = f.read()

start_index = text.find('THE MYSTERIOUS ISLAND')
end_index = text.find('End of the Project Gutenberg')
text = text[start_index:end_index]
char_set = set(text)
print(len(text))
print(len(char_set))

1112350
80


In [9]:
# как обычно будем кодировать буквы числами

chars_sorted = sorted(char_set)
charToInt = {c:i for i,c in enumerate(chars_sorted)}
char_array = np.array(chars_sorted)

text_encoded = np.array([charToInt[c] for c in text], dtype=np.int32)

ds_text_encoded = tf.data.Dataset.from_tensor_slices(text_encoded)

In [10]:
length = 40
chunk_size = length + 1

ds_chunks = ds_text_encoded.batch(chunk_size, drop_remainder=True)

def split_input_target(chunk): # разбиваем последовательность на вход и выход как показано на картинке
    input_seq = chunk[:-1]
    target_seq = chunk[1:]
    return input_seq, target_seq

ds_seqs = ds_chunks.map(split_input_target)

In [11]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

ds = ds_seqs.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True),
        tf.keras.layers.Dense(vocab_size) # выход - вероятности каждого символа как возможного следующего символа
    ])

    return model

charset_size = len(chars_sorted)
embedding_dim = 256
rnn_units = 512

tf.random.set_seed(42)

model = build_model(charset_size, embedding_dim, rnn_units)
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [13]:
model.compile(optimizer='adam', loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True))
model.fit(ds, epochs=20)

Epoch 1/20
424/424 ━━━━━━━━━━━━━━━━━━━━ 42s 95ms/step - loss: 2.5816
Epoch 2/20
424/424 ━━━━━━━━━━━━━━━━━━━━ 37s 86ms/step - loss: 1.6900
Epoch 3/20
424/424 ━━━━━━━━━━━━━━━━━━━━ 37s 86ms/step - loss: 1.4742
Epoch 4/20
424/424 ━━━━━━━━━━━━━━━━━━━━ 38s 89ms/step - loss: 1.3743
Epoch 5/20
424/424 ━━━━━━━━━━━━━━━━━━━━ 39s 90ms/step - loss: 1.3079
Epoch 6/20
424/424 ━━━━━━━━━━━━━━━━━━━━ 36s 84ms/step - loss: 1.2628
Epoch 7/20
424/424 ━━━━━━━━━━━━━━━━━━━━ 38s 90ms/step - loss: 1.2278
Epoch 8/20
424/424 ━━━━━━━━━━━━━━━━━━━━ 38s 89ms/step - loss: 1.2003
Epoch 9/20
424/424 ━━━━━━━━━━━━━━━━━━━━ 37s 86ms/step - loss: 1.1725
Epoch 10/20
424/424 ━━━━━━━━━━━━━━━━━━━━ 37s 86ms/step - loss: 1.1507
Epoch 11/20
424/424 ━━━━━━━━━━━━━━━━━━━━ 38s 89ms/step - loss: 1.1290
Epoch 12/20
424/424 ━━━━━━━━━━━━━━━━━━━━ 38s 88ms/step - loss: 1.1137
Epoch 13/20
424/424 ━━━━━━━━━━━━━━━━━━━━ 36s 85ms/step - loss: 1.0937
Epoch 14/20
424/424 ━━━━━━━━━━━━━━━━━━━━ 36s 85ms/step - loss: 1.0789
Epoch 15/20
424/424 ━━━━━━━━━

In [14]:
# так как задача этого кода - генерация текста, схожего с полученным, то если мы всё время будем генерировать следующий символ как наиболее вероятный, то будем получать один и тот же текст
# поэтому, лучше выбирать не наиболее вероятный символ, а символ, который попался, то есть мы выбираем символы с той вероятностью, с которой они в итоге были получены

logits = [[1., 1., 3.]]
print('Вероятности:', tf.math.softmax(logits).numpy()[0])
samples = tf.random.categorical(logits, num_samples=10)
tf.print(samples.numpy())

Вероятности: [0.10650697 0.10650697 0.786986  ]
array([[2, 2, 2, 2, 2, 2, 0, 2, 0, 0]])


In [ ]:
def sample(model, starting_str, len_generated_text=500, max_input_length=40, scale_factor=1.0): # процесс создания новой строки: задаётся начальная строка, берётся n её последних символов и на их основе генерируется новый символ, затем уже для полученной строки операция повторяется и так пока не достигнем желаемую длину строки
    incoded_input = [charToInt[s] for s in starting_str]
    encoded_input = tf.reshape(incoded_input, (1, -1))

    generated_str = starting_str

    # model.reset_state()
    for i in range(len_generated_text):
        logits = model(encoded_input)
        logits = tf.squeeze(logits, 0)

        scaled_logits = logits * scale_factor # чем больше коэффициент, тем предсказуемее результат, чем меньше, тем больше случайности и новизны
        new_char_indx = tf.random.categorical(
            scaled_logits, num_samples=1)
        
        new_char_indx = tf.squeeze(new_char_indx)[-1].numpy()    

        generated_str += str(char_array[new_char_indx])
        
        new_char_indx = tf.expand_dims([new_char_indx], 0)
        encoded_input = tf.concat(
            [encoded_input, new_char_indx],
            axis=1)
        encoded_input = encoded_input[:, -max_input_length:]

    return generated_str

In [20]:
print(sample(model, starting_str='The island'))

The island was enchored from the blood of your just could helpon, he risking our voice; and in a south-pine of saluting, cave and
passed down on each threw and round they
jagined wealth used of doubt upbility, employed again, this
anxious I crat.

This could not but a boat! Neb and Pencroft
skilled themselves in to return to the dirpline, and then were first. This which would be near the box. But he
was rachinguishing again, the document then saw a dozen lock
they had been an enemy of which it was shiddes
